In [ ]:
!pip install datasets


In [ ]:
from datasets import list_datasets, load_dataset


In [ ]:
imdb=load_dataset("imdb")

In [ ]:
imdb.keys()

In [ ]:
imdb["train"].column_names

In [ ]:
imdb["train"][0]

In [ ]:
imdb["train"].shape

In [ ]:
imdb["test"].shape

In [ ]:
imdb["train"].features

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [ ]:
tokenizer.is_fast

In [ ]:
type(tokenizer)

In [ ]:
tokenizer.model_max_length

In [ ]:
tokenizer.vocab_size

In [ ]:
X_train=imdb["train"]["text"]
y_train=imdb["train"]["label"]
X_test=imdb["test"]["text"]
y_test=imdb["test"]["label"]

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

In [ ]:
print(type(X_train))
print(type(y_train))


In [ ]:
train_encodings=tokenizer(X_train, max_length=128, truncation=True, padding=True)
test_encodings=tokenizer(X_test, max_length=128, truncation=True, padding=True)

In [ ]:
import tensorflow as tf

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

In [ ]:
train_dataset=train_dataset.shuffle(len(X_train)).batch(16)
test_dataset=test_dataset.batch(16)

In [ ]:
from transformers import TFAutoModelForSequenceClassification


In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf

# Load the pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define the optimizer using the transformers library
optimizer, _ = create_optimizer(
    init_lr=3e-5,
    num_train_steps=10000,  # Change this to the actual number of training steps
    num_warmup_steps=1000   # Change this to the actual number of warmup steps
)

# Define the loss and metrics
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Print the model summary
model.summary()


In [ ]:
# model=TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# opt=tf.keras.optimizers.Adam(learning_rate=3e-5)
# loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics=tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

# model.compile(optimizer=opt, loss=loss, metrics=metrics)
# model.summary()

In [ ]:
model.fit(train_dataset, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  37/1563 [..............................] - ETA: 5:16:25 - loss: 0.6962 - accuracy: 0.4905

In [ ]:
test_loss, test_acc=model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")